<a href="https://colab.research.google.com/github/Lesliec87/Amazon_Vine_Analysis/blob/main/vine_review_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-30 20:54:37--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.86MB/s    in 0.2s    

2022-06-30 20:54:37 (5.86 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32787517| RED72VWWCOS7S|B008HDQYLQ|     348668413|Garden Weasel Gar...| Lawn and Garden|          1|            2|          8|   N|                Y|            One Star|I don't hate the ...| 2015-08-31|
|         US|   16374060| RZHWQ208LTEPV|B005OBZBD6|     264704759|10 Foot Mc4 Solar...| Lawn and Garden|          5|    

In [15]:
totalvotes_20_df = df.filter("total_votes>=20")
totalvotes_20_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   39823685| RQQ3KVTU5TJ4I|B00005A3L1|     436617004|Bounty Hunter TK4...| Lawn and Garden|          5|           24|         29|   N|                Y|Payed for itself ...|I purchase this m...| 2015-08-31|
|         US|   35875101|R3FELXWV9T5CWE|B00VQVPRH8|     733961147|Multi-Purpose Boo...| Lawn and Garden|          2|    

In [16]:
helpful_50_df = totalvotes_20_df.filter("helpful_votes/total_votes>=.50")
helpful_50_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   39823685| RQQ3KVTU5TJ4I|B00005A3L1|     436617004|Bounty Hunter TK4...| Lawn and Garden|          5|           24|         29|   N|                Y|Payed for itself ...|I purchase this m...| 2015-08-31|
|         US|   35875101|R3FELXWV9T5CWE|B00VQVPRH8|     733961147|Multi-Purpose Boo...| Lawn and Garden|          2|    

In [17]:
vine_y_review_df = helpful_50_df.filter(helpful_50_df["vine"] == "Y")
vine_y_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   50842604|R28DXTC3JQ9IY1|B00O06YMGI|     949367908|Char-Broil Kettle...| Lawn and Garden|          4|           24|         26|   Y|                N|Consider Before B...|Everyone knows th...| 2015-08-23|
|         US|   46343881|R3AFZKLQXATHBU|B00SMRXYCE|     726092120|Char-Broil Deluxe...| Lawn and Garden|          5|    

In [18]:
vine_n_review_df = helpful_50_df.filter(helpful_50_df["vine"] == "N")
vine_n_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   39823685| RQQ3KVTU5TJ4I|B00005A3L1|     436617004|Bounty Hunter TK4...| Lawn and Garden|          5|           24|         29|   N|                Y|Payed for itself ...|I purchase this m...| 2015-08-31|
|         US|   35875101|R3FELXWV9T5CWE|B00VQVPRH8|     733961147|Multi-Purpose Boo...| Lawn and Garden|          2|    

In [26]:
total_reviews = df.count()
print(total_reviews)

2557288


In [24]:

total_y_reviews = vine_y_review_df.count() 
print(total_y_reviews)


386


In [25]:
total_n_reviews = vine_n_review_df.count() 
print(total_n_reviews)

48717


In [27]:
star_5_rating = vine_y_review_df.filter(vine_y_review_df["star_rating"] == 5)
print(star_5_rating.count())

176


In [28]:
star_5_n_rating = vine_n_review_df.filter(vine_n_review_df["star_rating"] == 5)
print(star_5_n_rating.count())

24026


In [29]:
percentage_Y_5_rating = star_5_rating.count()/total_y_reviews * 100
print(percentage_Y_5_rating)

45.59585492227979


In [30]:
percentage_N_5_rating = star_5_n_rating.count()/total_n_reviews * 100
print(percentage_N_5_rating)

49.317486708951705
